# ClusterEnsemble demonstration
Some example usage of how to build up a dataframe of galaxy cluster properties, including NFW halo profiles. Each cluster is treated as an individual, meaning we track its individual mass and redshift, and other properties. This is useful for fitting a stacked weak lensing profile, for example, where you want to avoid fitting a single average cluster mass.

In [ ]:
from __future__ import absolute_import, division, print_function

%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
from astropy import units
from matplotlib import pyplot as plt

#import seaborn as sns; sns.set() 

In [ ]:
from clusters import ClusterEnsemble

### Create a ClusterEnsemble object by passing in a numpy array (or list) of redshifts

In [ ]:
z = np.array([0.1,0.2,0.3])
c = ClusterEnsemble(z)
c.describe

### Display what we have so far
Below the DataFrame (which so far only contains the cluster redshifts), we see the default assumptions for the power-law slope and normalization that will be used to convert richness $N_{200}$ to mass $M_{200}$. We'll see how to change those parameters below.

In [ ]:
c.show()

In [ ]:
c.dataframe

In [ ]:
c.n200

### Add richness values to the dataframe
This step will also generate $M_{200}$, $r_{200}$, $c_{200}$, scale radius $r_s$, and other parameters, assuming the scaling relation given below.

In [ ]:
n200 = np.ones(3)*20.
c.n200 = n200
c.show()

### Access any column of the dataframe as an array
Notice that [astropy units](http://docs.astropy.org/en/stable/units/) are present for the appropriate columns.

In [ ]:
print('z: \t', c.z)
print('n200: \t', c.n200)
print('r200: \t', c.r200)
print('m200: \t', c.m200)
print('c200: \t', c.c200)
print('rs: \t', c.rs)

### If you don't want units, you can get just the values

In [ ]:
c.r200.value

### Change the redshifts or richness values
These changes will propogate to all redshift-dependant or richness-dependant cluster attributes, as appropriate.

In [ ]:
c.z = np.array([0.4,0.5,0.6])
c.show()

In [ ]:
c.n200 = [20,30,40]
c.show()

### Change the parameters in the mass-richness relation
Either or both of the keyword parameters "slope" and "norm" can be passed to the update_massrichrelation() method.

In [ ]:
c.update_massrichrelation(slope = 1.5)
c.show()

### Show basic table
Perhaps we don't want the fancy pandas formatting on our table, or maybe we're not working in the Jupyter notebook.

In [ ]:
c.show(notebook = False)

## Calculate $\Sigma(r)$ and $\Delta\Sigma(r)$ for NFW model
First select the radial bins in units of Mpc.

In [ ]:
rmin, rmax = 0.1, 5. #Mpc
nbins = 50
rbins = np.logspace(np.log10(rmin), np.log10(rmax), num = nbins)
#rbins

In [ ]:
%timeit c.calc_nfw(rbins)
sigma = c.sigma_nfw
dsigma = c.deltasigma_nfw

### There is now a Python implentation of the NFW calculations 
Set the keyword parameter "use_c = False" to use python only. Currently, the Python version is only implemented for perfectly centered halos. Note that this method is significantly faster than the C code, for the perfectly centered case (because it suboptimally writes/reads to disc in the latter case). Stand by for the miscentering offset timing comparison...

In [ ]:
%timeit c.calc_nfw(rbins, use_c = False)
sigma_py = c.sigma_nfw
dsigma_py = c.deltasigma_nfw

In [ ]:
#check the results match
np.testing.assert_allclose(sigma_py, sigma, rtol = 10**-4)
np.testing.assert_allclose(dsigma_py, dsigma, rtol = 10**-4)

In [ ]:
#sigma_py

In [ ]:
sigma

In [ ]:
for rich, profile in zip(c.n200,c.deltasigma_nfw):
    plt.plot(rbins, profile, label='$N_{200}=$ '+str(rich))
plt.xscale('log')
plt.legend(fontsize=20)

plt.xlim(rbins.min(), rbins.max())
plt.xlabel('$r\ [\mathrm{Mpc}]$', fontsize=20)
plt.ylabel('$\Delta\Sigma(r)\ [\mathrm{M}_\mathrm{sun}/\mathrm{pc}^2]$', fontsize=20)
plt.title('(Centered) Differential Surface Mass Density', fontsize=20)

In [ ]:
for rich, profile in zip(c.n200,c.sigma_nfw):
    plt.plot(rbins, profile, label='$N_{200}=$ '+str(rich))
plt.xscale('log')
plt.legend(fontsize=20)

plt.xlim(rbins.min(), rbins.max())
plt.xlabel('$r\ [\mathrm{Mpc}]$', fontsize=20)
plt.ylabel('$\Sigma(r)\ [\mathrm{M}_\mathrm{sun}/\mathrm{pc}^2]$', fontsize=20)
plt.title('(Centered) Surface Mass Density', fontsize=20)

# Calculate Miscentered NFW Profiles
First select the offsets in units of Mpc. The offset values parameterize the width of the Gaussian distribution of offsets, and is $\sigma_\mathrm{off}$ in Equation 11 of [Ford et al 2015](http://arxiv.org/abs/1409.3571).

In [ ]:
offsets = np.array([0.09,0.09,0.09])
c.calc_nfw(rbins, offsets=offsets)

In [ ]:
#print(c.deltasigma_offset)
#print(c.sigma_offset)

In [ ]:
for rich, profile in zip(c.n200,c.deltasigma_offset):
    plt.plot(rbins, profile, label='$N_{200}=$ '+str(rich))
plt.xscale('log')
plt.legend(fontsize=20)

plt.xlim(rbins.min(), rbins.max())
plt.xlabel('$r\ [\mathrm{Mpc}]$', fontsize=20)
plt.ylabel('$\Delta\Sigma^\mathrm{off}(r)\ [\mathrm{M}_\mathrm{sun}/\mathrm{pc}^2]$', 
           fontsize=20)
plt.title('Miscentered Differential Surface Mass Density', fontsize=20)

### To Do: 
- fix bug sometimes giving Inf in first bin of smoothed profiles
- replace smd_nfw.c with cython version
- write more tests
- use decorators (@property, @setter, @deleter) instead of update_z(), for example
- option to pass in a $M_{prelim}$ and $M_{200} = a \times$ $M_{prelim}$ relation